In [ ]:
import mne
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from metadata import *

print(subject_list)
biosemi_layout.plot()
plt.close()

In [ ]:
# COMPUTING ICA AND SAVE THE ICA DATA

ica_method = 'fastica' # other options: 'infomax', 'picard'
max_iter = 10000 # maximum iteration for fitting
decim = 8 # increment for selecting each nth time slice

for subject in subject_list: 
    print('##### Processing subject', subject, '...')

    print('# Processing raw data...')
    raw = mne.io.read_raw_fif(
        filtered_files[subject], 
        preload = True
    )
    raw.set_montage(biosemi_layout, raise_if_subset = False)

    print('# Fitting ICA...')
    ica = mne.preprocessing.ICA(
        method = ica_method,
        max_iter = max_iter
    )
    ica.fit(
        raw, 
        picks = EEG_channels+EXG_channels, 
        decim = decim
    )

    print('# Saving the ICA data...')
    ica.save(ica_files[subject])

print('***** ALL DONE *****')

In [ ]:
# VISUALIZE ALL COMPONENTS IN SCALP TOPOGRAPHY

for subject in subject_lists: 
    print('##### Processing subject', subject, '...')

    print('# Reading the ICA data...')
    ica = mne.preprocessing.read_ica(ica_files[subject])

    print('# Visualizing ICA components...')
    figure = ica.plot_components(show = False)

    print('# Saving figures...')
    figure[0].savefig(ica_viz_dir+subject+'_'+ica_method+'_component_0-19.png')
    figure[1].savefig(ica_viz_dir+subject+'_'+ica_method+'_component_20-36.png')

print('***** ALL DONE *****')

In [ ]:
# FOR EACH SUBJECT, VISUALIZE DIAGNOSTICS PER CHANNEL

for subject in subject_list: 
    print('##### Processing subject', subject, '...')

    print('# Reading Raw and ICA data...')
    raw = mne.io.read_raw_fif(filtered_files[subject], preload=True)
    ica = mne.preprocessing.read_ica(ica_files[subject])

    print('# Plotting diagnostics for all channels...')
    figure = ica.plot_properties(
        raw, 
        picks = 'all', 
        show = False
    )

    print('# Saving figures...')
    for ic in range(len(figure)): 
        index = '0'*(3-len(str(ic)))+str(ic)
        figure[ic].savefig(ica_viz_dir+subject+'_'+'ICA'+index+'.png')
    
    plt.close('all') # prevent figures printed out in the console

print('***** ALL DONE *****')